## Character level BPE (Byte Pair Encoding) with word boundaries

Instead of using the regular alphabet we wish to find better set of symbols (character sequences that appear togetehr frequently) and treat them as single units.

### Intuition:

If "at" appears 200 times in your text then why represent it using 'a' and 't' separately. Instead make [at] a new symbol.

### learnings:

- implemented char level BPE (an algorithm similar to the tokenizer for GPT-2. It uses Byte level BPE with no explicit word endings)
- For our model we need to ensure that "cat btb tb" is not parsed as {'c':1, 'a':1, 'tb' : 3} i.e. word endings are to be taken seriously.
- for this split words at space and added a '_' to mark end
- then the frequency counter was run on each word basis and the most frequent across the whole text was merged in vocabulary
- as the number of merges (each merge is 1 iteration updating the vocabulary) increases -- we are able to capture more symbols and thus reduce the tokens in the test text encodding.
- once we have trained (generated the vocabulary on the basis of statistics) we can encode any given text. the compression varies depenind on the distribution match between the train text and testing text.
- should be noted that character level restricts us to english only (the language of training)
- byte level would make this language independent (what GPT2 and modern LLMs do). They can even tokenize emojis!


### Overall:

BPE discovers linguistic structure automatically just by statistics!

### Explanation

BPE EXAMPLE: "aaabcaabbd" for 1 merge

Input: "aaabcaabbd_"
Tokens: ['a','a','a','b','c','a','a','b','b','d','_']  (11 tokens)

Count adjacent pairs:
  ('a','a'): 3  ← MOST FREQUENT
  ('a','b'): 2
  ('b','c'): 1
  ('b','b'): 1
  ('b','d'): 1
  ('d','_'): 1

Merge most frequent: ('a','a') → 'aa'

Output: ['aa','a','b','c','aa','b','b','d','_']  (9 tokens)

Compression: 11 → 9 tokens


### For text with words and spaces

BPE in 4 Steps:

1. Add word boundaries:     "the cat" → ["the_", "cat_"]

2. Find most frequent pair:  ('t','h') appears 100x

3. Merge everywhere:         ('t','h','e','_') → ('th','e','_')

4. Repeat N times → Learn N new symbols

Result: Learned vocabulary of subword units

In [119]:
import re

import requests

url = "https://raw.githubusercontent.com/yanranzhao/n-gram-language-identification/master/en-the-little-prince.txt"
raw_text = requests.get(url).text

def preprocess_text(text):
    """Clean text and add word boundary markers."""
    text = text.lower()
    text = text.replace('\n', ' ')
    text = re.sub(r'([.,!?;:\'\"-])', r' \1 ', text)
    text = ' '.join(text.split())
    
    words = text.split()
    return [word + '_' for word in words]


In [116]:
import re
from collections import Counter


def get_vocab(words):
    """
    Get vocabulary from words.
    Each word is represented as a tuple of tokens.
    
    words: list of strings like ['the_', 'cat_']
    returns: dict mapping word tuple to frequency
    """
    vocab = {}
    for word in words:
        # Each word starts as tuple of characters
        word_tuple = tuple(word)
        vocab[word_tuple] = vocab.get(word_tuple, 0) + 1
    return vocab


def get_pairs(vocab):
    """
    Count all adjacent pairs across all words in vocab.
    
    vocab: {('t','h','e','_'): 5, ('c','a','t','_'): 3}
    returns: Counter of pairs
    """
    pairs = Counter()
    
    for word, freq in vocab.items():
        for i in range(len(word) - 1):
            pairs[(word[i], word[i+1])] += freq
    
    return pairs


def merge_vocab(pair, vocab):
    """
    Merge all occurrences of pair in vocab.
    
    pair: ('t', 'h')
    vocab: {('t','h','e','_'): 5} 
    returns: {('th','e','_'): 5}
    """
    new_vocab = {}
    
    # Create the merged token
    merged = ''.join(pair)
    
    for word, freq in vocab.items():
        # Merge pair in this word
        new_word = []
        i = 0
        while i < len(word):
            if i < len(word) - 1 and word[i] == pair[0] and word[i+1] == pair[1]:
                new_word.append(merged)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        
        new_vocab[tuple(new_word)] = freq
    
    return new_vocab


def train_bpe(words, num_merges, verbose=True):
    """
    Train BPE.
    
    words: list of strings ['the_', 'cat_', ...]
    num_merges: number of merge operations
    returns: list of merge operations [(pair, merged_token), ...]
    """
    # Get initial vocabulary (character level)
    vocab = get_vocab(words)
    
    merges = []
    
    if verbose:
        print(f"Training on {len(words)} words")
        print(f"Unique words: {len(vocab)}")
        print(f"Learning {num_merges} merges...\n")
    
    for i in range(num_merges):
        pairs = get_pairs(vocab)
        
        if not pairs:
            break
        
        # Most frequent pair
        best_pair = max(pairs, key=pairs.get)
        freq = pairs[best_pair]
        
        if freq < 2:
            break
        
        # Merge it
        vocab = merge_vocab(best_pair, vocab)
        merged_token = ''.join(best_pair)
        merges.append((best_pair, merged_token))
        
        if verbose and (i < 5 or i >= num_merges - 3):
            print(f"Merge {i+1}: {best_pair} → '{merged_token}' (freq: {freq})")
    
    if verbose:
        print(f"\nLearned {len(merges)} merges")
    
    return merges


def encode_word(word, merges):
    """
    Apply BPE merges to a single word.
    
    word: string like 'running_'
    merges: learned merge operations
    returns: list of tokens ['r', 'u', 'nn', 'ing_']
    """
    # Start with characters
    tokens = list(word)
    
    # Apply each merge in order
    for pair, merged in merges:
        i = 0
        new_tokens = []
        while i < len(tokens):
            if i < len(tokens) - 1 and tokens[i] == pair[0] and tokens[i+1] == pair[1]:
                new_tokens.append(merged)
                i += 2
            else:
                new_tokens.append(tokens[i])
                i += 1
        tokens = new_tokens
    
    return tokens


def encode_text(text, merges):
    """
    Tokenize text using learned BPE merges.
    
    text: raw string
    merges: learned merge operations
    returns: list of tokens
    """
    words = preprocess_text(text)
    
    all_tokens = []
    for word in words:
        word_tokens = encode_word(word, merges)
        all_tokens.extend(word_tokens)
    
    return all_tokens



In [121]:

words = preprocess_text(raw_text)
merges = train_bpe(words, num_merges=500, verbose=True)
print(f"We have learnt frequency based symbols by {len(merges)} merges.\n")

# Show learned vocabulary
print("\n" + "=" * 70)
print("LEARNED MERGES")
print("=" * 70)
for i, (pair, merged) in enumerate(merges, 1):
    print(f"{i:2d}. {pair} → '{merged}'")
    
test_text = """
The little prince said that the little fox told the little prince about the rose, and the rose was special because the little prince had tamed the rose, and taming means creating ties, and the little prince understood that the rose was unique among all the roses in the world because the little prince had spent time with the rose, watering the rose and protecting the rose from the wind, and the little prince realized that what makes the desert beautiful is that somewhere it hides a well, and what makes the stars beautiful is the invisible flower that blooms somewhere among them, and the little prince learned that it is only with the heart that one can see rightly, because what is essential is invisible to the eye, and the fox said to the little prince that the time the little prince had spent with the rose made the rose important, and the little prince would forever be responsible for the rose because the little prince had tamed the rose, and the baobabs must be pulled up as soon as they can be distinguished from the rosebushes, because if the baobabs grow too large they will destroy the little prince's planet, and the little prince visited many planets including the planet of the king who claimed to rule over everything, and the planet of the conceited man who wanted everyone to admire him, and the planet of the drunkard who drank to forget that he was ashamed of drinking, and the planet of the businessman who counted the stars and claimed to own them, and the planet of the lamplighter who faithfully lit and extinguished his lamp even though his planet spun faster and faster, and the narrator met the little prince in the desert when the narrator's plane crashed, and the narrator drew pictures for the little prince including a sheep in a box because the little prince wanted a sheep to eat the baobabs but not to eat the rose with thorns, and the little prince asked the narrator to draw a muzzle for the sheep so the sheep could not eat the rose, and eventually the little prince returned to his planet by allowing the snake to bite him, and the narrator still looks up at the stars wondering if the sheep has eaten the rose or if the rose is safe behind her glass globe, and the narrator tells us that all grown-ups were once children but few of them remember it.
"""

print("We can now encode any given new text")
tokens = encode_text(test_text, merges)

print(f"Original Length: {len(test_text)} characters")
print(f"Original Word count: {len(test_text.split())} words")

print(f"New Tokens: {tokens}")
print(f"Compression: {len(test_text)} chars → {len(tokens)} tokens\n")
print(f"Compression: {(1 - len(tokens)/len(test_text))*100:.1f}%")


Training on 1705 words
Unique words: 477
Learning 500 merges...

Merge 1: ('e', '_') → 'e_' (freq: 243)
Merge 2: ('t', 'h') → 'th' (freq: 170)
Merge 3: ('t', '_') → 't_' (freq: 150)
Merge 4: ('d', '_') → 'd_' (freq: 138)
Merge 5: ('s', '_') → 's_' (freq: 136)
Merge 498: ('m', 'or') → 'mor' (freq: 2)
Merge 499: ('mor', 'e_') → 'more_' (freq: 2)
Merge 500: ('l', 'at') → 'lat' (freq: 2)

Learned 500 merges
We have learnt frequency based symbols by 500 merges.


LEARNED MERGES
 1. ('e', '_') → 'e_'
 2. ('t', 'h') → 'th'
 3. ('t', '_') → 't_'
 4. ('d', '_') → 'd_'
 5. ('s', '_') → 's_'
 6. ('n', '_') → 'n_'
 7. ('e', 'r') → 'er'
 8. ('y', '_') → 'y_'
 9. ('.', '_') → '._'
10. ('i', 'n') → 'in'
11. ('a', 'n') → 'an'
12. (',', '_') → ',_'
13. ('i', '_') → 'i_'
14. ('a', '_') → 'a_'
15. ('o', '_') → 'o_'
16. ('th', 'e_') → 'the_'
17. ('o', 'u') → 'ou'
18. ('e', 'd_') → 'ed_'
19. ('f', '_') → 'f_'
20. ('in', 'g') → 'ing'
21. ('r', 'e') → 're'
22. ('h', 'a') → 'ha'
23. ('i', 's_') → 'is_'
24. ('